In [2]:
import pandas as pd

data = pd.read_csv("groceriesSynthetic.tsv", sep = "\t")
df = pd.DataFrame(data)
df_original = pd.DataFrame(data)

In [3]:
#Checking if all the values in currency are EUR
print(df["currency"].nunique() == "EUR")

False


In [4]:
rows_with_other_currencies = df[df["currency"] != "EUR"]
print(rows_with_other_currencies)

         id formattedDate               merchantName  \
804     704    06-03-2023                        NaN   
1362   1384    25-06-2023                        NaN   
1806   1381    04-04-2023                        NaN   
2530     18    16-03-2023                        NaN   
3066   1110    01-12-2022                        NaN   
...     ...           ...                        ...   
96983   477    15-04-2023                        NaN   
97047  1763    16-08-2023                        NaN   
97815  1218    05-06-2023                        NaN   
97902  1446    30-08-2023                        NaN   
99425   985    15-07-2023  TALLINN anyone Debrashire   

                                          remittanceInfo   value currency  
804    292384******9276 either JARVE SELVERI ISETEENI...  -10.74      USD  
1362   (..3895) carry any WoltItaemerenkatu somebody new    -5.7      SEK  
1806   (..6464) always sign protect everybody improve...  -22.44      SEK  
2530   (..5274) scene p

In [5]:
pip install CurrencyConverter


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
from currency_converter import CurrencyConverter

c = CurrencyConverter()

def currency_to_EUR(value, currency):
    if currency != "EUR":
        try:
            conversion = c.convert(value, currency, "EUR")
            return round(conversion, 2)
        except Exception as e:
            print(f"Error converting {value} {currency} to EUR: {e}")
    return value

df["value"] = df.apply(lambda row: currency_to_EUR(row["value"], row["currency"]), axis=1)

In [7]:
rows_with_other_currencies = df[df["currency"] != "EUR"]
print(rows_with_other_currencies)

         id formattedDate               merchantName  \
804     704    06-03-2023                        NaN   
1362   1384    25-06-2023                        NaN   
1806   1381    04-04-2023                        NaN   
2530     18    16-03-2023                        NaN   
3066   1110    01-12-2022                        NaN   
...     ...           ...                        ...   
96983   477    15-04-2023                        NaN   
97047  1763    16-08-2023                        NaN   
97815  1218    05-06-2023                        NaN   
97902  1446    30-08-2023                        NaN   
99425   985    15-07-2023  TALLINN anyone Debrashire   

                                          remittanceInfo  value currency  
804    292384******9276 either JARVE SELVERI ISETEENI... -10.06      USD  
1362   (..3895) carry any WoltItaemerenkatu somebody new  -0.49      SEK  
1806   (..6464) always sign protect everybody improve...  -1.93      SEK  
2530   (..5274) scene prepa

In [8]:
df = df.drop(columns="currency")

In [9]:
def findMerchant(x):
    if isinstance(x, str):
        mer = ""
        li = x.split()
        i = 0
        while li[i].isupper():
            mer + " " + li[i]
        return mer

In [10]:
#finding all the merchants in the merchantName column
merchantsFromDf = df["merchantName"].apply(lambda x: x.split()[0] if isinstance(x, str) and x.split()[0].isupper() else None).dropna().unique().tolist()
print(merchantsFromDf)

['MAKSIMARK', 'OU', 'RIMI', 'SELVER', 'KAUPLUS', 'KAUBAMAJA', 'KONSUM', 'AS', 'MAXIMA', 'VALD', 'POOD', 'COOP', 'TOIDUKAUBAD', 'TALLINN', 'R-KIOSK', 'PRISMA', 'ISETEEN', 'OÜ', 'A', 'STOCKMANN', 'LIDL', 'KESKUS', 'GROSSI', 'TURU', 'ICA', 'TURG', 'ISETEENINDUS', 'MAKSEKESKUS', 'PAIDE', 'MEIE', 'KURESSAARE', 'MARKET', 'TARTU', 'WWW.RIMI.EE', 'A&O', 'R', 'VIRU', 'KARDLA', 'VILJANDI', 'PARNU', 'NARVA', 'KRISTIINE', 'JOHVI', 'BOLT', 'NARVESEN', 'KOHTLA-JARVE', 'TOIT', 'POLVA', 'MAARDU', 'SIKUPILLI', 'ROCCA', 'EEDEN', 'PUHVET', 'GET', 'BLOOM', 'APOLLO']


In [11]:
finalMerchantList = ['MAKSIMARK', 'MAKSIMARKET','RIMI','KAUBAMAJA','SELVER', 'KONSUM','MAXIMA', 'COOP','PRISMA','STOCKMANN','LIDL', 'GROSSI', 'ICA', 'MEIE TOIDUKAUBAD',
                    'A&O', 'BOLT MARKET', 'SIKUPILLI', 'BLOOM']

In [12]:
def process_merchant_name_column(row):
    if pd.isnull(row["merchantName"]):
        for merchant in finalMerchantList:
            if merchant in row["remittanceInfo"].upper():
                return merchant
    else:
        list = row["merchantName"].split()
        if list[0].isupper() and list[0] in finalMerchantList:
            return list[0]
        else:
            return None

In [13]:
df["merchantName"] = df.apply(process_merchant_name_column, axis=1)

In [14]:
df = df.dropna(subset=['merchantName'])

In [15]:
df.head(20)

,id,formattedDate,merchantName,remittanceInfo,value
0,1269,24-07-2023,MAKSIMARK,567176******6366 toward information MAKSIMARKE...,-13.59
1,319,14-06-2023,LIDL,157466******5153 pay LIDL own TARTU sound behi...,-0.89
3,1340,20-02-2023,MAKSIMARK,24/04/1997 resource kaart...878232 VILJANDI MA...,-12.9
6,400,10-03-2023,ICA,551163******9009 which ICA AB/TARTU LOUNAKESK ...,-5.1
7,1310,04-01-2023,MAXIMA,Kaart:411726******8003 human seven community a...,-17.35
8,184,07-09-2023,RIMI,576266******5911 return RIMI/TATARI RIMI ISETE...,-4.28
9,1292,09-01-2023,KONSUM,368085******4980 difficult act KONSUM ISETEENI...,5
11,535,09-05-2023,SELVER,29/05/1985 traditional kaart...609240 huge SEL...,-21.86
14,690,05-03-2023,KONSUM,146990******6058 light IHASTE KONSUM marriage ...,-1.29
17,322,01-07-2022,SELVER,(..1988) short simply stand SELVER ISETEENINSO...,-34.08


In [16]:
df.shape

(66667, 5)

In [17]:
print(df[df["merchantName"]=="KURESSAARE"])

Empty DataFrame
Columns: [id, formattedDate, merchantName, remittanceInfo, value]
Index: []


In [1]:
# IMPORTIME re, ET SAAKS SPILTTIDA MITME CONDITIONI PÕHJAL
import re

In [19]:
# TEKITAME UUE TULBA NIMEGA cardNumber NING PANEME VAIKEVÄÄRTUSEKS NONE

df["cardNumber"] = None

In [21]:
# TEKITAME UUED TULBAD NIMEDEGA city, location NING PANEME VAIKEVÄÄRTUSTEKS NONE

df["city"] = None
df["location"] = None

In [23]:
# TEKITAME UUED TULBAD NIMEDEGA selfCheckout, date NING PANEME VAIKEVÄÄRTUSTEKS NONE

df["selfCheckout"] = None
df["date"] = None

In [25]:
# IMPORTIME datetime, ET SAAKSIME TEKITADA KUUPÄEVI

from datetime import *

In [27]:
#  TEKITAME LISTI EESTI LINNADEST

eestiLinnad = [
    "Võru", "Võhma", "Viljandi", "Valga", "Türi", "Tõrva", "Tartu", "Tapa", "Tamsalu", "Tallinn",
    "Suure-Jaani", "Sindi", "Sillamäe", "Saue", "Räpina", "Rapla", "Rakvere", "Püssi", "Pärnu",
    "Põlva", "Põltsamaa", "Paldiski", "Paide", "Otepää", "Narva-Jõesuu", "Narva", "Mõisaküla",
    "Mustvee", "Maardu", "Loksa", "Lihula", "Kärdla", "Kuressaare", "Kunda", "Kohtla-Järve",
    "Kiviõli", "Kilingi-Nõmme", "Keila", "Kehra", "Karksi-Nuia", "Kallaste", "Jõhvi", "Jõgeva",
    "Haapsalu", "Elva", "Antsla", "Abja-Paluoja"
]

# KUNA SAIN SELLE LISTI CHAT-ILT SIIS TEEME KÕIK UPPERCASE NING TEKITAME KA VARIANDID IAGST KUS LÕPUS ON MAA (PÕLVA -> PÕLVAMAA)

for i in range(0, len(eestiLinnad)):
    eestiLinnad[i] = eestiLinnad[i].upper()
    eestiLinnad.append(eestiLinnad[i]+"MAA")

eestiLinnad

['VÕRU',
 'VÕHMA',
 'VILJANDI',
 'VALGA',
 'TÜRI',
 'TÕRVA',
 'TARTU',
 'TAPA',
 'TAMSALU',
 'TALLINN',
 'SUURE-JAANI',
 'SINDI',
 'SILLAMÄE',
 'SAUE',
 'RÄPINA',
 'RAPLA',
 'RAKVERE',
 'PÜSSI',
 'PÄRNU',
 'PÕLVA',
 'PÕLTSAMAA',
 'PALDISKI',
 'PAIDE',
 'OTEPÄÄ',
 'NARVA-JÕESUU',
 'NARVA',
 'MÕISAKÜLA',
 'MUSTVEE',
 'MAARDU',
 'LOKSA',
 'LIHULA',
 'KÄRDLA',
 'KURESSAARE',
 'KUNDA',
 'KOHTLA-JÄRVE',
 'KIVIÕLI',
 'KILINGI-NÕMME',
 'KEILA',
 'KEHRA',
 'KARKSI-NUIA',
 'KALLASTE',
 'JÕHVI',
 'JÕGEVA',
 'HAAPSALU',
 'ELVA',
 'ANTSLA',
 'ABJA-PALUOJA',
 'VÕRUMAA',
 'VÕHMAMAA',
 'VILJANDIMAA',
 'VALGAMAA',
 'TÜRIMAA',
 'TÕRVAMAA',
 'TARTUMAA',
 'TAPAMAA',
 'TAMSALUMAA',
 'TALLINNMAA',
 'SUURE-JAANIMAA',
 'SINDIMAA',
 'SILLAMÄEMAA',
 'SAUEMAA',
 'RÄPINAMAA',
 'RAPLAMAA',
 'RAKVEREMAA',
 'PÜSSIMAA',
 'PÄRNUMAA',
 'PÕLVAMAA',
 'PÕLTSAMAAMAA',
 'PALDISKIMAA',
 'PAIDEMAA',
 'OTEPÄÄMAA',
 'NARVA-JÕESUUMAA',
 'NARVAMAA',
 'MÕISAKÜLAMAA',
 'MUSTVEEMAA',
 'MAARDUMAA',
 'LOKSAMAA',
 'LIHULAMAA',
 'KÄRDLA

In [29]:
# SIIT NÄHA MILLINE ON remittanceInfo SPILT

counter = 0
for index, row in df.iterrows():
    if counter <= 50:
        rowRemittanceSeperatedList = re.split(r'[ ,/]', row["remittanceInfo"])
        print(rowRemittanceSeperatedList)
        counter += 1

['567176******6366', 'toward', 'information', 'MAKSIMARKET', 'outside', 'VILJANDIMAA', '']
['157466******5153', 'pay', 'LIDL', 'own', 'TARTU', 'sound', 'behind', 'TARTUMAA', '', 'dinner']
['24', '04', '1997', 'resource', 'kaart...878232', 'VILJANDI', 'MAKSIMARKET', 'VILJANDIMAA', '', 'EST', 'carry']
['551163******9009', 'which', 'ICA', 'AB', 'TARTU', 'LOUNAKESK', 'six', 'TARTUMAA', '', 'TAR']
['Kaart:411726******8003', 'human', 'seven', 'community', 'act', '-', 'Maxima', 'bank', '-', 'three', 'community']
['576266******5911', 'return', 'RIMI', 'TATARI', 'RIMI', 'ISETE', 'travel', 'TALLINN']
['368085******4980', 'difficult', 'act', 'KONSUM', 'ISETEENINDUSKESKVALJAK', '1KARDLA9']
['29', '05', '1985', 'traditional', 'kaart...609240', 'huge', 'SELVERI', 'ISETEENI', 'TALLINN', 'EST', 'team']
['146990******6058', 'light', 'IHASTE', 'KONSUM', 'marriage', 'TARTUMAA', '', 'buy']
['(..1988)', 'short', 'simply', 'stand', 'SELVER', 'ISETEENINSOBRA', 'wrong', 'TARTU', 'LINN', 'animal', 'thought']
[

In [31]:
# KUI VÕIMALIK SIIS TEKITAME remittanceInfost KUUPÄEVA
def getDate(rowList):
    try:
        day = rowList[0]
        month = rowList[1]
        year = rowList[2]
        if len(day) == 2 and len(month) == 2 and len(year) == 4:
            purchaseDate = date(int(year), int(month), int(day))
            formatted_date = purchaseDate.strftime('%d-%-m-%-Y')  # Format date as 'YYYY/M/D'
            return (True, formatted_date)
        else:
            return (False, False)
    except:
        return (False, False)
    
# KONTROLLIME KAS remittanceInfo SISALDAB MÄRGET, ET TEGU OLI ISETEENINDUSKASSAGA
def isSelfCheckout(elementFromRowAsList):
    if "ISETE" in elementFromRowAsList:
        return True
    else:
        return False    

# KÄIME LÄBI KÕIK DF READ NING EXTRACTIME SEALT KAARDINUMBRI JA/VÕI ISETEENINDUSKASSA MÄRGISTUSE JA/VÕI LINNA JA/VÕI KUUPÄEVA
# NING PANEME VASTAVAD ANDMED VASTAVATESSE TULPADESSE
for index, row in df.iterrows():
    rowRemittanceSeperatedList = re.split(r'[ ,/]', str(row["remittanceInfo"]))
    for i in range(0,len(rowRemittanceSeperatedList), 1):
        currentElement = str(rowRemittanceSeperatedList[i])
        if (currentElement != "" and currentElement != " " and currentElement != "-"):
            if (currentElement.count("*") > 1 or (currentElement[0] == "(" and currentElement[-1] == ")") or "kaart..." in currentElement):
                df.at[index, "cardNumber"] = currentElement
            if isSelfCheckout(currentElement):
                df.at[index, "selfCheckout"] = True
            if rowRemittanceSeperatedList[i].upper() in eestiLinnad:
                df.at[index, "city"] = currentElement.upper()
            dateCheck = getDate(rowRemittanceSeperatedList)
            if dateCheck[0] == True:
                df.at[index, "date"] = dateCheck[1]
            
# TO-DO
# A) cardNumber PUHTAKS, ET EI OLEKS kaart...983921 -> ....983921
#    IDEAALIS VÕIKSID KÕIK KAARDINUMBRID OLLA SAMA PIKAD
# B) - 

In [33]:
df.head(40)

,id,formattedDate,merchantName,remittanceInfo,value,cardNumber,city,location,selfCheckout,date
0,1269,24-07-2023,MAKSIMARK,567176******6366 toward information MAKSIMARKE...,-13.59,567176******6366,VILJANDIMAA,None,None,None
1,319,14-06-2023,LIDL,157466******5153 pay LIDL own TARTU sound behi...,-0.89,157466******5153,TARTUMAA,None,None,None
3,1340,20-02-2023,MAKSIMARK,24/04/1997 resource kaart...878232 VILJANDI MA...,-12.9,kaart...878232,VILJANDIMAA,None,None,24-4-1997
6,400,10-03-2023,ICA,551163******9009 which ICA AB/TARTU LOUNAKESK ...,-5.1,551163******9009,TARTUMAA,None,None,None
7,1310,04-01-2023,MAXIMA,Kaart:411726******8003 human seven community a...,-17.35,Kaart:411726******8003,None,None,None,None
8,184,07-09-2023,RIMI,576266******5911 return RIMI/TATARI RIMI ISETE...,-4.28,576266******5911,TALLINN,None,True,None
9,1292,09-01-2023,KONSUM,368085******4980 difficult act KONSUM ISETEENI...,5,368085******4980,None,None,True,None
11,535,09-05-2023,SELVER,29/05/1985 traditional kaart...609240 huge SEL...,-21.86,kaart...609240,TALLINN,None,True,29-5-1985
14,690,05-03-2023,KONSUM,146990******6058 light IHASTE KONSUM marriage ...,-1.29,146990******6058,TARTUMAA,None,None,None
17,322,01-07-2022,SELVER,(..1988) short simply stand SELVER ISETEENINSO...,-34.08,(..1988),TARTU,None,True,None


In [35]:
# EEMALDAME UUESTI "MAA" city LÕPUST KUNA NEID TEGELIKULT EI OLE VAJA. NEID OLI VAJA VAID TUVASTAMISEKS
# VA. PÕLTSAMAA SEST SELLE LINNA NIMI LÕPPEBKI "MAA"-GA JA SEE ON AINUKE SELLINE LINN EESTIS
for index, row in df.iterrows():
    if  row["city"] is not None and row["city"].endswith("MAA") and row["city"] != "PÕLTSAMAA":
        df.at[index, "city"] = row["city"][:-3]

In [37]:
df.head(50)

,id,formattedDate,merchantName,remittanceInfo,value,cardNumber,city,location,selfCheckout,date
0,1269,24-07-2023,MAKSIMARK,567176******6366 toward information MAKSIMARKE...,-13.59,567176******6366,VILJANDI,None,None,None
1,319,14-06-2023,LIDL,157466******5153 pay LIDL own TARTU sound behi...,-0.89,157466******5153,TARTU,None,None,None
3,1340,20-02-2023,MAKSIMARK,24/04/1997 resource kaart...878232 VILJANDI MA...,-12.9,kaart...878232,VILJANDI,None,None,24-4-1997
6,400,10-03-2023,ICA,551163******9009 which ICA AB/TARTU LOUNAKESK ...,-5.1,551163******9009,TARTU,None,None,None
7,1310,04-01-2023,MAXIMA,Kaart:411726******8003 human seven community a...,-17.35,Kaart:411726******8003,None,None,None,None
8,184,07-09-2023,RIMI,576266******5911 return RIMI/TATARI RIMI ISETE...,-4.28,576266******5911,TALLINN,None,True,None
9,1292,09-01-2023,KONSUM,368085******4980 difficult act KONSUM ISETEENI...,5,368085******4980,None,None,True,None
11,535,09-05-2023,SELVER,29/05/1985 traditional kaart...609240 huge SEL...,-21.86,kaart...609240,TALLINN,None,True,29-5-1985
14,690,05-03-2023,KONSUM,146990******6058 light IHASTE KONSUM marriage ...,-1.29,146990******6058,TARTU,None,None,None
17,322,01-07-2022,SELVER,(..1988) short simply stand SELVER ISETEENINSO...,-34.08,(..1988),TARTU,None,True,None


In [39]:
#RASMUSE ESIMENE

df['merchantName'] = df['merchantName'].replace('ICA', 'RIMI')
df['merchantName'] = df['merchantName'].replace('KONSUM', 'COOP')
df['merchantName'] = df['merchantName'].replace('MAKSIMARK', 'COOP')
df['merchantName'] = df['merchantName'].replace('MAKSIMARKET', 'COOP')

In [41]:

#RASMUSE TEINE

df['value'] = pd.to_numeric(df['value'], errors='coerce') #Conveting shoppin cart values to numeric

average_values = df.groupby('merchantName')['value'].mean() #Calculating average shopping cart values by store
print(average_values)

merchantName
A&O                -14.912647
BLOOM               -1.750571
COOP               -16.400579
GROSSI             -15.540681
KAUBAMAJA          -14.187237
LIDL               -15.146752
MAXIMA             -15.824094
MEIE TOIDUKAUBAD   -24.935000
PRISMA             -16.363286
RIMI               -17.140231
SELVER             -16.311279
SIKUPILLI          -13.311818
STOCKMANN          -15.647270
Name: value, dtype: float64
